# TODO
> PICK UP HERE: **implement model end-to-end ASAP using Dataset, saving model state every N epochs, etc.**
- See todo list in OneNote
- Once working, make the saving model state logic generic so I can use with any model

In [18]:
from datatype_recovery.models.dataset import TypeSequenceDataset

data_params = {
    'experiment_runs': [
        '/home/cls0027/test_builds/astera.exp/rundata/run1',
        '/home/cls0027/test_builds/coreutils.exp/rundata/run1',
    ],
    'copy_data': False,
}
dataset = TypeSequenceDataset('model0_dataset', data_params, max_hops=3)

input_params will be IGNORED in favor of saved .json file (model0_dataset/raw/input_params.json)


In [155]:
# TODO: split test/train on BINARY boundary to prevent functions/vars from crossing
# -> I have varid tuples, so I can get (rungid, binid) from that and use it to split
#   - extract (idx, rungid, binid) for each data point and put into a dataframe
#   - groupby/count dataframe by (rungid/binid) and come up with split that is close to target %
#   - collect indices for each data point within each subset (test/train/valid)
#   - quick/simple validate that all (rungid/binid) combos are unique to a subset
#
# [(i, x.varid[0], x.varid[1]) for i, x in enumerate(dataset[10000:10100])][:5]

from torch_geometric.loader import DataLoader
from torch.utils.data import Subset

# tiny subset to start
trainset_indices = list(range(12))
testset_indices = list(range(100,112))

train_set = Subset(dataset, trainset_indices)
test_set = Subset(dataset, testset_indices)

BATCH_SIZE = 4

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

# EXAMPLE: the shuffled slice changes each time, but dataset[0] stays consistent
# print(dataset[:10].shuffle()[0].varid)
# print(dataset[0].varid)

In [156]:
for step, data in enumerate(train_loader):
    print(f'Step {step+1}')
    print(f'Num graphs in batch: {data.num_graphs}')
    print(data)
    print()

Step 1
Num graphs in batch: 4
DataBatch(x=[47, 47], edge_index=[2, 94], y=[4, 20], varid=[4], batch=[47], ptr=[5])

Step 2
Num graphs in batch: 4
DataBatch(x=[86, 47], edge_index=[2, 164], y=[4, 20], varid=[4], batch=[86], ptr=[5])

Step 3
Num graphs in batch: 4
DataBatch(x=[67, 47], edge_index=[2, 130], y=[4, 20], varid=[4], batch=[67], ptr=[5])



In [31]:


# TEMP: just split randomly for now
len(dataset)

44021

In [100]:
rungid, bid, _, _, _ = dataset[0].varid
[(x.varid[0], x.varid[1]) for x in dataset[10000:10100]][:5]

[(1, 81.0), (1, 81.0), (1, 81.0), (1, 81.0), (1, 82.0)]

In [107]:
[(i, x.varid[0], x.varid[1]) for i, x in enumerate(dataset[10000:10100])][:5]

indices = [10000, 10001, 10002, 10003, 10004]
from torch.utils.data import Subset

ss = Subset(dataset, indices)

In [137]:
# [(x.varid[0], x.varid[1]) for x in ss.shuffle()]
loader = DataLoader(ss, shuffle=True)

In [138]:
[(x.varid[0][0], x.varid[0][1]) for x in loader]

[(1, 81.0), (1, 82.0), (1, 81.0), (1, 81.0), (1, 81.0)]

In [160]:
from pathlib import Path
import pandas as pd
pd.read_csv(Path('/home/cls0027/test_builds/coreutils.exp/rundata/run1/binaries.csv')).BinaryId
df = pd.read_csv(Path('/home/cls0027/test_builds/coreutils.exp/rundata/run1/locals.csv'))
# df.BinaryId = df.BinaryId.astype('int64')
# df.loc[:,'NEW_COL'] = pd.Series([1e3]*len(df), dtype='int64')
df.BinaryId

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
         ...  
11580    116.0
11581    116.0
11582    116.0
11583    116.0
11584    116.0
Name: BinaryId, Length: 11585, dtype: float64

In [83]:
df.NEW_COL

0        1000
1        1000
2        1000
3        1000
4        1000
         ... 
11580    1000
11581    1000
11582    1000
11583    1000
11584    1000
Name: NEW_COL, Length: 11585, dtype: int64

In [86]:
df = pd.read_csv(Path('/home/cls0027/test_builds/coreutils.exp/rundata/run1/47.pathchk/functions.csv'))
df.BinaryId

0      47
1      47
2      47
3      47
4      47
       ..
101    47
102    47
103    47
104    47
105    47
Name: BinaryId, Length: 106, dtype: int64